In [ ]:
!pip install spacy-transformers

In [ ]:
import pandas as pd
from spacy.tokens import DocBin
import spacy

## Fetching the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/прога/Проект/test.csv')
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/прога/Проект/train.csv')

In [ ]:
train.head(3)

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4


In [ ]:
X_train = train['full_text'].tolist()
y_train = train['score'].tolist()

len(X_train), len(y_train)
y_train.pop(0)
len(X_train), len(y_train)
# y_train = [i.replace('\n', '').split(',') for i in y_train]
train_y_df = pd.DataFrame(y_train, columns=['score'])
train_x_df = pd.DataFrame(X_train, columns=['full_texts'])
train_y_df.label = train_y_df.score.astype(str)
train_y_df.reset_index(inplace=True)
train_x_df.reset_index(inplace=True)
data = pd.concat((train_x_df, train_y_df), axis=1)
data.drop(['index'], axis=1, inplace=True)
data

<ipython-input-5-42917676d930>:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  train_y_df.label = train_y_df.score.astype(str)


,full_texts,score
0,Many people have car where they live. The thin...,3.0
1,I am a scientist at NASA that is discussing th...,4.0
2,People always wish they had the same technolog...,4.0
3,"We all heard about Venus, the planet without a...",3.0
4,"Dear, State Senator\n\nThis is a letter to arg...",4.0
...,...,...
17302,"the story "" The Challenge of Exploing Venus "" ...",4.0
17303,Technology has changed a lot of ways that we l...,2.0
17304,If you don't like sitting around all day than ...,1.0
17305,"In ""The Challenge of Exporing Venus,"" the auth...",2.0


In [ ]:
data = data.sample(frac=1, random_state=0)
train_data = data[0:10000]
dev_data = data[10000:12000]
test_data = data[12000:17307]

train_data.reset_index(inplace=True)
dev_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [ ]:
data.score.unique()

array([ 3.,  1.,  4.,  5.,  2.,  6., nan])

## Preprocessing

In [ ]:
data.shape

(17307, 2)

In [ ]:
nlp = spacy.load("en_core_web_sm")

def lemmatize(text):
    words = []
    doc = nlp(text)
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
train['clean_texts'] = train['full_text'].apply(lemmatize)

In [ ]:
train = train.dropna(subset=['score'])

In [ ]:
train

,essay_id,full_text,score,clean_texts
0,000d118,Many people have car where they live. The thin...,3,people car live thing know use car alot thing ...
1,000fe60,I am a scientist at NASA that is discussing th...,3,scientist NASA discuss face mar explain face l...
2,001ab80,People always wish they had the same technolog...,4,People wish technology see movie good new piec...
3,001bdc0,"We all heard about Venus, the planet without a...",4,hear Venus planet oxygen earthquake erupt volc...
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,dear State Senator letter argue favor keep Ele...
...,...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2,story Challenge Exploing Venus informative pie...
17303,ffddf1f,Technology has changed a lot of ways that we l...,4,technology change lot way live today nowadays ...
17304,fff016d,If you don't like sitting around all day than ...,2,like sit day great opportunity participate sea...
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1,challenge Exporing Venus author suggest study ...


In [ ]:
data = data.dropna(subset=['score'])

In [ ]:
data['clean_texts'] = train['clean_texts']

<ipython-input-19-4cda6cad3849>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_texts'] = train['clean_texts']


In [ ]:
data = data.sample(frac=1, random_state=0)
train_data = data[0:10000]
dev_data = data[10000:12000]
test_data = data[12000:17307]

train_data.reset_index(inplace=True)
dev_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [ ]:
def convert(data, outfile):
    nlp = spacy.blank("en")
    db = DocBin()

    data['score'] = data['score'].astype(str)
    categories = data['score'].unique().tolist()

    # categories = [1, 2, 3, 4, 5, 6]
    for len_num in range(len(data)):
        doc = nlp.make_doc(data['clean_texts'][len_num])
        doc.cats = {category: 0 for category in categories}
        doc.cats[data['score'][len_num]] = 1
        db.add(doc)
    db.to_disk(outfile)

In [ ]:
convert(train_data, 'train.spacy')
convert(dev_data, 'dev.spacy')
convert(test_data, 'test.spacy')

<ipython-input-28-cc73cd961001>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score'] = data['score'].astype(str)
<ipython-input-28-cc73cd961001>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score'] = data['score'].astype(str)
<ipython-input-28-cc73cd961001>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --gpu-id 0 --paths.train ./train.spacy  --paths.dev ./dev.spacy --output model

ℹ Saving to output directory: model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       49.83    0.50
  0     200          28.63       48.90    0.49
  0     400          28.09 

In [ ]:
!python -m spacy evaluate model/model-best/ --output metrics.json ./test.spacy --gpu-id 0

ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   50.95 
SPEED                 361875


=========================== Textcat F (per label) ===========================

          P        R       F
5.0    0.00     0.00    0.00
2.0   26.71   100.00   42.15
3.0    0.00     0.00    0.00
4.0    0.00     0.00    0.00
1.0    0.00     0.00    0.00
6.0    0.00     0.00    0.00


======================== Textcat ROC AUC (

In [ ]:
with open('/content/input/x_test.txt') as f:
    score_data = f.readlines()

In [ ]:
score_data_clean = [i.replace('\n', '') for i in score_data]

In [ ]:
series_score_data_clean = pd.Series(score_data_clean)

In [ ]:
import spacy
import spacy_transformers

In [ ]:
nlp = spacy.load('./model/model-best')
doc=nlp("History is made: 10 new UK attractions for day trips and short breaks")
print(doc.cats['1.0'])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


0.49924999475479126
